[분류] 여행 보험 패키지 상품을 구매할 확률 값을 구하시오 / 리더보드 제출용  
예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)  
평가: roc-auc 평가지표  

In [146]:
import pandas as pd

train = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-1-train.csv')
test = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-1-test.csv')

In [147]:
train.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,10000,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No,0
1,10001,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No,0
2,10002,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No,1
3,10003,33,Government Sector,Yes,650000.0,6,1,No,No,1
4,10004,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes,1


In [148]:
test.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,0,32,Government Sector,Yes,650000.0,5,0,No,No
1,1,27,Government Sector,Yes,500000.0,5,0,No,No
2,2,32,Government Sector,Yes,350000.0,2,0,No,No
3,3,26,Private Sector/Self Employed,Yes,1400000.0,6,0,No,Yes
4,4,27,Private Sector/Self Employed,No,350000.0,3,0,No,No


In [149]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1490 non-null   int64  
 1   Age                  1490 non-null   int64  
 2   Employment Type      1490 non-null   object 
 3   GraduateOrNot        1490 non-null   object 
 4   AnnualIncome         1486 non-null   float64
 5   FamilyMembers        1490 non-null   int64  
 6   ChronicDiseases      1490 non-null   int64  
 7   FrequentFlyer        1490 non-null   object 
 8   EverTravelledAbroad  1490 non-null   object 
 9   TravelInsurance      1490 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 116.5+ KB


In [150]:
train.isnull().sum()

id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           4
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64

In [151]:
test.isnull().sum()

id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           3
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [152]:
test = test.iloc[:,1:]
target = train.iloc[:,-1]
train = train.iloc[:,1:-1]

In [153]:
target

0       0
1       0
2       1
3       1
4       1
       ..
1485    0
1486    0
1487    1
1488    0
1489    0
Name: TravelInsurance, Length: 1490, dtype: int64

In [154]:
# 결측치 - 평균으로 채우기

train = train.fillna(train.AnnualIncome.mean())
test = test.fillna(test.AnnualIncome.mean())

In [155]:
train.isnull().sum()

Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [156]:
test.isnull().sum()

Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [157]:
train['Employment Type'].unique(), test['Employment Type'].unique()

(array(['Private Sector/Self Employed', 'Government Sector'], dtype=object),
 array(['Government Sector', 'Private Sector/Self Employed',
        'Casual employment'], dtype=object))

In [158]:
train = train.drop(columns='Employment Type')
test = test.drop(columns='Employment Type')

In [159]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [160]:
for obj in train.dtypes.index:
    if train[obj].dtype == 'object':
        train[obj] = le.fit_transform(train[obj])
        test[obj] = le.transform(test[obj])

In [161]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  1490 non-null   int64  
 1   GraduateOrNot        1490 non-null   int64  
 2   AnnualIncome         1490 non-null   float64
 3   FamilyMembers        1490 non-null   int64  
 4   ChronicDiseases      1490 non-null   int64  
 5   FrequentFlyer        1490 non-null   int64  
 6   EverTravelledAbroad  1490 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 81.6 KB


In [162]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  497 non-null    int64  
 1   GraduateOrNot        497 non-null    int64  
 2   AnnualIncome         497 non-null    float64
 3   FamilyMembers        497 non-null    int64  
 4   ChronicDiseases      497 non-null    int64  
 5   FrequentFlyer        497 non-null    int64  
 6   EverTravelledAbroad  497 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 27.3 KB


- 모델 학습

In [163]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [164]:
X_train, X_val, y_train, y_val = train_test_split(train, target,
                                                 test_size=0.2, random_state=42)

In [172]:
rf = RandomForestClassifier(n_estimators=300, max_depth=5)
lgbm = LGBMClassifier(n_estimators=300, max_depth=5)

In [173]:
rf.fit(X_train, y_train)
pred1 = rf.predict(X_val)

In [174]:
lgbm.fit(X_train, y_train)
pred2 = lgbm.predict(X_val)

In [175]:
pred2

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1])

- 모델 평가

In [176]:
from sklearn.metrics import roc_auc_score

In [177]:
roc_auc_score(y_val, pred1), roc_auc_score(y_val, pred2)

(0.7785138248847927, 0.7784178187403993)

- 제출

In [178]:
pred = rf.predict(test)

In [179]:
pred

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [181]:
output = pd.DataFrame({'pred':pred})

In [182]:
output.to_csv('1234567.csv', index=False)

In [183]:
pd.read_csv('1234567.csv')

,pred
0,0
1,0
2,0
3,1
4,0
...,...
492,0
493,1
494,0
495,1
